In [1]:
import pandas as pd

PATH = "../../DS-Unit-2-Applied-Modeling/data/dota/"

In [21]:
%%time
matches_small_chunk = pd.read_csv(PATH + 'matches_small.csv', chunksize=10000)
i = 0

for chunk in matches_small_chunk:
    df = chunk[chunk.human_players == 10].reset_index()
    df.to_csv(path_or_buf=PATH + "full_players_games.csv", mode='a')
    i += df.shape[0]
display(i)

1911704

Wall time: 4min 46s


In [25]:
%%time
matches_small_chunk = pd.read_csv(PATH + 'matches_small.csv', chunksize=10000)
i = 0

for chunk in matches_small_chunk:
    df = chunk[chunk.human_players == 10]
    df.to_csv(path_or_buf=PATH + "full_players_games (1).csv", mode='a')
    i += df.shape[0]
display(i)

1911704

Wall time: 4min 20s


In [28]:
%%time

import random

random.seed(0)

n = 1911704 #number of records in file
s = 20000 #desired sample size
filename = PATH + "full_players_games (1).csv"
skip = sorted(random.sample(range(1, n),n-s))
df = pd.read_csv(filename, skiprows=skip)
df

Wall time: 22.5 s


,Unnamed: 0,match_id,match_seq_num,radiant_win,start_time,duration,tower_status_radiant,tower_status_dire,barracks_status_radiant,barracks_status_dire,...,engine,picks_bans,parse_status,chat,objectives,radiant_gold_adv,radiant_xp_adv,teamfights,version,pgroup
0,278.0,2041649420,1798304581,f,1451488879,2162,0,1982,0,63,...,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{""0"":{""account_id"":148759085,""hero_id"":87,""pla..."
1,336.0,2041624514,1798304747,f,1451488292,2584,4,1830,3,63,...,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{""0"":{""account_id"":266975308,""hero_id"":98,""pla..."
2,486.0,2041613089,1798305033,f,1451488012,3063,48,452,12,51,...,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{""0"":{""account_id"":4294967295,""hero_id"":14,""pl..."
3,618.0,2041612309,1798305261,t,1451487994,2731,1972,0,63,0,...,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{""0"":{""account_id"":4294967295,""hero_id"":36,""pl..."
4,705.0,2041630487,1798305356,t,1451488431,2655,1956,0,63,0,...,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{""0"":{""account_id"":114486262,""hero_id"":11,""pla..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20190,1959510.0,2306490763,2021151279,f,1461109160,2517,6,1972,3,63,...,1,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,"{""0"":{""account_id"":4294967295,""hero_id"":104,""p..."
20191,1959511.0,2306496027,2021151287,f,1461109544,2100,256,2046,48,63,...,1,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,"{""0"":{""account_id"":4294967295,""hero_id"":39,""pl..."
20192,1959512.0,2306500208,2021151293,t,1461109918,1728,2047,0,63,0,...,1,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,"{""0"":{""account_id"":4294967295,""hero_id"":18,""pl..."
20193,1959513.0,2306480548,2021151298,t,1461108202,3274,1844,0,63,0,...,1,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,"{""0"":{""account_id"":4294967295,""hero_id"":70,""pl..."


In [31]:
df.drop("Unnamed: 0", axis=1).radiant_win.apply(lambda x : True if x == 't' else False).value_counts(normalize=True)

True     0.514434
False    0.485566
Name: radiant_win, dtype: float64

In [33]:
pd.read_csv(PATH + "full_players_games (1).csv", usecols=['radiant_win'], squeeze=True).apply(lambda x : True if x == 't' else False).value_counts(normalize=True)

True     0.516706
False    0.483294
Name: radiant_win, dtype: float64